In [1]:
import ETL_functions as etf
import numpy as np
import pandas as pd
import ast

# steam_games.json

In [2]:
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json", lines=True,dtype=str)#lo cargamos como str ya que la mayoria de sus columnas requieren 
#este formato ahorrando asi :
# tener que transformar individualmente las columnas que contienen listas 
# cambiar el valor float de 0 y 1 early access a boolean
#a su vez esto transformara todos los nulls a cadenas de texto las cuales deberemos acceder atravez de el valor "None"(string)
#por lo que ya no podremos utilizar las funciones para el manejo de nulos de pandas

In [3]:
df_sg.shape #las dimensiones de el dataframe son de 120445 filas y 13 columnas

(120445, 13)

In [4]:
df_sg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   publisher     120445 non-null  object
 1   genres        120445 non-null  object
 2   app_name      120445 non-null  object
 3   title         120445 non-null  object
 4   url           120445 non-null  object
 5   release_date  120445 non-null  object
 6   tags          120445 non-null  object
 7   reviews_url   120445 non-null  object
 8   specs         120445 non-null  object
 9   price         120445 non-null  object
 10  early_access  120445 non-null  object
 11  id            120445 non-null  object
 12  developer     120445 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


## correccion los problemas en los datos encontrados durante la primer parte de el eda

### eliminacion de duplicados

In [5]:
df_sg.duplicated().value_counts()#tiene 88309 registros duplicados los cuales no nos interesan

True     88309
False    32136
Name: count, dtype: int64

In [6]:
df_sg.drop_duplicates(inplace=True)#dropeo duplicados lo que incluye la mayoria de columnas con valores nulls

In [7]:
df_sg.shape #las dimensiones de el dataframe sin duplicados son de 32136 filas y 13 columnas

(32136, 13)

In [8]:
df_sg.head(5)#(para asegurarnos que ahora que borro todas las columnas repetidas, solo nos dejo una unica fila con solo valores None la que dropearemos)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,None,None,None
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400,彼岸领域


In [9]:
df_sg.drop(0,inplace=True)#con esto borramos la unica columna que no poseia datos que quedaba

## ahora asignaremos lo dtype correspondientes a cada campo

### "price" str a float

In [10]:
lista = etf.valores_no_numericos(df_sg["price"])# esta funcion la hice en ETL_funciones
print(lista)

['Free To Play', 'Free to Play', 'None', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']


In [11]:
#quito de la lista estos 2 valores que no se podian convertir a numerico pero contiene su precio numerico en la cadena
lista.remove("Starting at $499.00")
lista.remove('Starting at $449.00')

In [12]:
#remplazo estas 2 cadenas por sus valores numericos
df_sg["price"] = df_sg["price"].replace("Starting at $499.00","499") 
df_sg["price"] = df_sg["price"].replace("Starting at $449.00","449") 
#mask = df_sg["price"].isin(lista)

In [13]:
lista #ahora la lista solo contiene valores que podria reemplazar por precio 0

['Free To Play',
 'Free to Play',
 'None',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Free to Try',
 'Free Movie',
 'Free to Use']

In [14]:
df_sg["price"] = df_sg["price"].replace(lista,"0")#cambio los valores string que equivalen a precio 0 por "0"

In [15]:
df_sg["price"] = df_sg["price"].astype(float)#cambiando el dtype a float

### "id" str a int

In [16]:
lista = etf.valores_no_numericos(df_sg["id"])
lista #los valores en la columna "id" que no pueden ser convertidos a int

['None']

In [17]:
mask = (df_sg["id"] == "None")#filtro que selecciona solo las columnas con "id" con valor "None"
df_sg = df_sg[~mask]# ~ sirve para dropear , aca estamos dropeando las filas en que tienen id none

In [18]:
df_sg["id"] = df_sg["id"].astype(int)

### "early-access"str a boolean

In [19]:
df_sg["early_access"].value_counts()#como ya esta dicho en la parte donde se carga la libreria al cargar todo el df como str se cambiaron de sus valores float
# a sus correspondientes valores booleanos en formato str y al parecer no poseia datos nulos o incorrectos

early_access
False    30186
True      1947
Name: count, dtype: int64

In [20]:
df_sg["early_access"] = df_sg["early_access"].astype(bool)# transformamos a boolean

## resultado final

    float(price),bool(early_access)int(id) y el resto de columnas siendo de tipo str

In [22]:
df_sg.info()#asi es como quedarian nuestros tipos de datos

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     32133 non-null  object 
 1   genres        32133 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         32133 non-null  object 
 4   url           32133 non-null  object 
 5   release_date  32133 non-null  object 
 6   tags          32133 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         32133 non-null  object 
 9   price         32133 non-null  float64
 10  early_access  32133 non-null  bool   
 11  id            32133 non-null  int32  
 12  developer     32133 non-null  object 
dtypes: bool(1), float64(1), int32(1), object(10)
memory usage: 3.1+ MB


## guardando el df procesado en un archivo parquet

In [23]:
df_sg.to_parquet(r"../app/processed_data/steam_games.parquet")#dentro de la carpeta de la api

<hr>

In [ ]:
# list items
reviews = []
# open file json
with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
# create data frame with data
df_user_reviews = pd.DataFrame(reviews)
# display df
df_user_reviews

<hr>

In [ ]:
# list items
reviews = []
# open file json
with open(r'../Datasets/users_items.json/australian_users_items.json', 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
# create data frame with data
df_user_items = pd.DataFrame(reviews)
# display df
df_user_items##88310 rows 5 columns #4 min

<hr>